In [ ]:
# import libraries 
import openai
import os
import json
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Set OpenAI API key from environment variable
openai.api_type = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = f"https://{os.getenv('OPENAI_SERVICE_NAME')}.openai.azure.com/"
openai.api_version = os.getenv("OPENAI_API_VERSION")

# Read the text-sample.json
with open('../data/text-sample.json', 'r', encoding='utf-8') as file:
    input_data = json.load(file)

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine="text-embedding-ada-002")
    embeddings = response['data'][0]['embedding']
    return embeddings


# Generate embeddings for title and content fields
for item in input_data:
    title = item['title']
    content = item['content']
    title_embeddings = generate_embeddings(title)
    content_embeddings = generate_embeddings(content)
    item['titleVector'] = title_embeddings
    item['contentVector'] = content_embeddings
    item['@search.action'] = 'upload'

# Output embeddings to docVectors.json file
with open("../output/docVectors.json", "w") as f:
    json.dump(input_data, f)
